# Estimação de Densidade por Kernel e Detecção de Outliers em Preços do Airbnb em Lisboa

## Objetivo
Este notebook estima a distribuição dos preços de acomodações do Airbnb em Lisboa usando **Kernel Density Estimation (KDE)** com kernel Gaussiano. A largura de banda é escolhida por **máxima verossimilhança leave-one-out (MLKDE)** e a densidade resultante é utilizada para identificar preços improváveis (outliers).

## 1. Título e objetivo
Este estudo busca:
- Estimar a densidade dos preços usando KDE com kernel Gaussiano.
- Escolher a largura de banda (bandwidth) que maximiza a log-verossimilhança leave-one-out (MLKDE).
- Usar a densidade estimada para destacar preços com baixa probabilidade (outliers).

## 2. Importação de bibliotecas
Carregamos as bibliotecas necessárias para manipulação de dados, cálculo numérico e visualização.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

plt.style.use("seaborn-v0_8-whitegrid")
%matplotlib inline


## 3. Carregamento e preparação dos dados
O conjunto de dados é o `listings.csv` do **Inside Airbnb** (Lisboa), salvo localmente como `data/listings_lisboa.csv`. A análise usa apenas a coluna de preços (`price`).

Passos de limpeza:
1. Remover linhas sem preço.
2. Converter strings de preço (por exemplo, `"$120.00"`) para valores numéricos.
3. Filtrar preços não positivos.
4. Criar `log_price = log(preço)`, o que reduz a influência de caudas longas e facilita a modelagem.

In [ ]:
data_path = Path("data/listings_lisboa.csv")
if not data_path.exists():
    raise FileNotFoundError(
        "Arquivo data/listings_lisboa.csv não encontrado. Baixe o listings.csv de Lisboa no Inside Airbnb e salve com esse nome."
    )

# Leitura e cópia do conjunto de dados original
raw_df = pd.read_csv(data_path)
if "price" not in raw_df.columns:
    raise KeyError("A coluna 'price' não foi encontrada no arquivo fornecido.")

# Limpeza da coluna de preço
prices = (
    raw_df["price"]
    .astype(str)
    .str.replace(r"[^0-9,\.]", "", regex=True)  # remove símbolos monetários e outros caracteres
    .str.replace(",", "", regex=False)  # remove separadores de milhar
)
prices = pd.to_numeric(prices, errors="coerce")

clean_df = raw_df.copy()
clean_df["price"] = prices
clean_df = clean_df.dropna(subset=["price"]).copy()
clean_df = clean_df[clean_df["price"] > 0].copy()
clean_df["log_price"] = np.log(clean_df["price"])
clean_df = clean_df.reset_index(drop=True)

print(f"Total de entradas após limpeza: {len(clean_df)}")
clean_df[["price", "log_price"]].head()


## 4. Análise exploratória simples
A seguir mostramos estatísticas descritivas e histogramas para `price` e `log_price`, destacando assimetria e caudas longas.

In [ ]:
estatisticas = clean_df[["price", "log_price"]].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
estatisticas


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].hist(clean_df["price"], bins=40, color="#4c72b0", edgecolor="black")
axes[0].set_title("Histograma de price")
axes[0].set_xlabel("Preço")
axes[0].set_ylabel("Frequência")

axes[1].hist(clean_df["log_price"], bins=40, color="#55a868", edgecolor="black")
axes[1].set_title("Histograma de log_price")
axes[1].set_xlabel("log(preço)")
axes[1].set_ylabel("Frequência")

plt.tight_layout()
plt.show()


## 5. Implementação do KDE e do MLKDE

**Kernel Density Estimation (KDE)** aproxima a função densidade de probabilidade dos dados. Para cada ponto de avaliação \(x\), a estimativa é dada por:

\[\hat{f}(x) = rac{1}{n h} \sum_{i=1}^{n} K\left( rac{x - x_i}{h} ight)\]

- \(K\) é o kernel Gaussiano (simétrico, suave e com integral 1).
- \(h\) (bandwidth) controla a suavização: valores pequenos produzem curvas muito onduladas; valores grandes podem esconder detalhes.

Para escolher \(h\), usamos **máxima verossimilhança leave-one-out (MLKDE)**:
- Para cada \(x_i\), calculamos \(\hat{f}_{-i}(x_i)\) excluindo o próprio ponto.
- Somamos \(\log(\hat{f}_{-i}(x_i))\) para obter a log-verossimilhança.
- O \(h\) ótimo maximiza essa soma.

In [ ]:
def gaussian_kernel(u: np.ndarray) -> np.ndarray:
    """Kernel Gaussiano padrão."""
    return (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * u ** 2)


def kde_density(x_grid: np.ndarray, data: np.ndarray, bandwidth: float) -> np.ndarray:
    """KDE univariado com kernel Gaussiano."""
    n = data.shape[0]
    scaled_diff = (x_grid[:, None] - data[None, :]) / bandwidth
    kernel_vals = gaussian_kernel(scaled_diff)
    density = kernel_vals.mean(axis=1) / bandwidth
    return density


def loo_log_likelihood(data: np.ndarray, bandwidth: float) -> float:
    """Log-verossimilhança leave-one-out para um bandwidth dado."""
    n = data.shape[0]
    if bandwidth <= 0:
        return -np.inf
    scaled_diff = (data[:, None] - data[None, :]) / bandwidth
    kernel_vals = gaussian_kernel(scaled_diff)
    np.fill_diagonal(kernel_vals, 0.0)
    density_i = kernel_vals.sum(axis=1) / ((n - 1) * bandwidth)
    density_i = np.maximum(density_i, 1e-12)  # evita log(0)
    return float(np.sum(np.log(density_i)))


log_prices = clean_df["log_price"].to_numpy()
std_log = np.std(log_prices, ddof=1) if len(log_prices) > 1 else 1.0

# Intervalo de busca para h
h_min = max(0.05, 0.1 * std_log)
h_max = max(h_min * 2, 1.5 * std_log)
h_values = np.linspace(h_min, h_max, 40)

log_liks = [loo_log_likelihood(log_prices, h) for h in h_values]
best_idx = int(np.argmax(log_liks))
best_h = h_values[best_idx]

print(f"Bandwidth ótimo (MLKDE): {best_h:.4f}")


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(h_values, log_liks, marker="o", color="#4c72b0")
ax.axvline(best_h, color="#c44e52", linestyle="--", label=f"h ótimo = {best_h:.4f}")
ax.set_xlabel("Bandwidth (h)")
ax.set_ylabel("Log-verossimilhança LOO")
ax.set_title("Busca do bandwidth por MLKDE")
ax.legend()
plt.show()


O valor de \(h\) que maximiza a log-verossimilhança oferece o melhor equilíbrio entre suavização e fidelidade aos dados segundo o critério LOO.

## 6. KDE final com o bandwidth ótimo
Usamos o \(h\) selecionado para estimar a densidade em um grid ao redor de `log_price` e comparamos a curva KDE com o histograma (densidade empírica).

In [ ]:
margin = 0.5 * std_log if std_log > 0 else 0.5
x_grid = np.linspace(log_prices.min() - margin, log_prices.max() + margin, 200)
density_grid = kde_density(x_grid, log_prices, best_h)

fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(clean_df["log_price"], bins=40, density=True, color="#c7e9c0", edgecolor="#4c72b0", alpha=0.8, label="Histograma (densidade)")
ax.plot(x_grid, density_grid, color="#1b9e77", linewidth=2.5, label="KDE com h ótimo")
ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade")
ax.set_title("Densidade estimada vs. histograma")
ax.legend()
plt.show()


A curva KDE suavizada acompanha a distribuição observada de `log_price`, revelando a forma geral da densidade e destacando possíveis regiões com menor probabilidade.

## 7. Detecção de outliers com base na densidade
Pontos com densidade estimada muito baixa indicam preços improváveis. Usaremos um critério simples: marcar como outlier qualquer observação cuja densidade estimada esteja abaixo do percentil 1% da densidade nos pontos amostrais.

In [ ]:
densidades_nos_pontos = kde_density(log_prices, log_prices, best_h)
limiar = np.percentile(densidades_nos_pontos, 1)
clean_df["kde_density"] = densidades_nos_pontos
clean_df["outlier"] = clean_df["kde_density"] < limiar

resumo_outliers = clean_df[["price", "log_price", "kde_density", "outlier"]]

print(f"Limiar de densidade (1%): {limiar:.6f}")
resumo_outliers.head()


Alguns exemplos de preços marcados como outliers (densidade muito baixa):

In [ ]:
outliers = clean_df[clean_df["outlier"]].sort_values("price", ascending=False)
print("Outliers com preços mais altos:")
display(outliers.head(5)[["price", "log_price", "kde_density"]])

print("
Outliers com preços mais baixos:")
display(outliers.sort_values("price").head(5)[["price", "log_price", "kde_density"]])


Os resultados devem refletir preços extremamente altos ou baixos em comparação com a massa principal dos dados.

## 8. Visualização dos outliers
Visualizamos a relação entre `log_price` e a densidade estimada, destacando os pontos marcados como outliers.

In [ ]:
cores = np.where(clean_df["outlier"], "#e84a5f", "#4c72b0")

fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(clean_df["log_price"], clean_df["kde_density"], c=cores, alpha=0.7, edgecolor="white", linewidth=0.6)
ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade estimada")
ax.set_title("Densidade por log(preço) com outliers destacados")
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(clean_df["price"], clean_df["kde_density"], c=cores, alpha=0.7, edgecolor="white", linewidth=0.6)
ax.set_xlabel("Preço")
ax.set_ylabel("Densidade estimada")
ax.set_title("Densidade por preço (escala original)")
ax.set_xscale("log")
plt.show()


Os pontos em vermelho mostram regiões onde a densidade é particularmente baixa, sinalizando preços atípicos. O uso da escala logarítmica ajuda a visualizar a dispersão quando há grande variação nos preços.

## 9. Conclusão
- Estimamos a densidade dos preços do Airbnb em Lisboa via KDE com kernel Gaussiano.
- Escolhemos o bandwidth por máxima verossimilhança leave-one-out (MLKDE), equilibrando suavização e aderência aos dados.
- A densidade estimada foi usada para marcar outliers como observações com densidade abaixo do percentil 1%.
- Observamos que a transformação logarítmica estabiliza a variabilidade e facilita a detecção de preços incomuns.
- Limitações: o critério de outlier é simples e univariado; análises futuras podem segmentar por bairro, tipo de acomodação ou incorporar variáveis adicionais para detecção mais robusta.